In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import os
from labellines import labelLines

In [ ]:
plt.style.use("presentation-dark.mplstyle")

In [ ]:
plt.rcParams["font.size"] = 25

In [ ]:
# "qdset" is a xarray.Dataset storing the (16, 50, 84) percentile values and mean and std for each model's time series into an effectively 3D array.
with xr.open_dataarray("percentiles_all.nc") as qdset:
    qmean = qdset.sel(q="mean")

hst_basedir = "lowZ-hst-data"
hst = dict()
for m in qdset["name"].data:
    f = os.path.join(hst_basedir, f"{m}_hst.nc")
    with xr.open_dataset(f) as dset:
        hst[m] = dset

In [ ]:
colors_dark = {
    "Srad": "w",
    "Sheat": "coral",
    "Scool": "aqua",
    "Snet": "lime",
    "SSN": "gold",
    "SKE": "yellowgreen",
    "Fout": "violet",
    "Gwork": "lightgray",
    "Sbox": "bisque",
    "Esink": "orange",
    "PE": "aqua",
    "PH_HI": "coral",
    "CR": "gold",
    "other_heat": "bisque",
    "CII": "aqua",
    "OI": "orange",
    "neb": "coral",
    "HI_Lya": "lime",
    "rec_cool": "gold",
    "CIE": "violet",
    "other_cool": "bisque",
}

outdir = "./lowZ-energetics-figures"
os.makedirs(outdir, exist_ok=True)

In [ ]:
plt.plot(-hst[m]["Esink"] / 228)

In [ ]:
def plot_energetics_build(hst):
    fig, axes = plt.subplots(1, 1, figsize=(10, 12), sharex=True, sharey=True)
    dt = 5
    tMyr = hst["tMyr"]
    window = int(dt / tMyr.diff(dim="time_code").median())
    hst["Gwork"] = hst["Gext"] + hst["Gsg"] + hst["Gtid"]
    hst["Sbox"] = hst["Rxy"] + hst["Mxy"]
    for key, ylabel in zip(
        ["Srad", "Sheat", "Scool", "Snet", "SSN", "Fout", "Gwork", "Sbox", "SKE"],
        [
            r"$\dot{S}_{\rm UV}$",
            r"$\mathcal{G}$",
            r"$\mathcal{L}$",
            r"$\mathcal{L}-\mathcal{G}$",
            r"$\dot{S}_{\rm SN}$",
            r"$\mathcal{F}_{\rm E, out}$",
            r"$G_{xy}$",
            r"$R_{xy}+M_{xy}$",
            r"$\dot{S}_{\rm diss}$",
            r"$\dot{S}_{\rm sink}$",
        ],
    ):
        yavg = hst[key].to_series().rolling(window, center=True, min_periods=1).mean()
        plt.plot(tMyr, yavg, color=colors_dark[key], label=ylabel)
        if key == "SKE":
            plt.fill_between(
                tMyr, yavg / 2, yavg * 2, color=colors_dark["SKE"], alpha=0.5, lw=0
            )
        plt.yscale("log")
        plt.ylim(1.0e-6, 1.0e4)

    plt.ylabel(r"Energy Source/Sink" + r"$\,[{L_\odot\,{\rm pc^{-2}}}]$")
    plt.setp(axes, xlabel=r"$t\,[{\rm Myr}]$")  # ,xlim=(200,500))
    plt.tight_layout()
    lines = axes.lines

    labelLines(
        align=False,
        zorder=2.5,
    )

In [ ]:
for m in hst:
    plot_energetics_build(hst[m])
    plt.title(m, fontsize="xx-small")
    plt.savefig(os.path.join(outdir, f"{m}-erate_tevol.png"), dpi=300)

In [ ]:
def plot_energetics_residual(hst):
    fig, axes = plt.subplots(1, 1, figsize=(10, 12), sharex=True, sharey=True)
    dt = 5
    tMyr = hst["tMyr"]
    window = int(dt / tMyr.diff(dim="time_code").median())
    hst["Gwork"] = hst["Gext"] + hst["Gsg"] + hst["Gtid"]
    hst["Sbox"] = hst["Rxy"] + hst["Mxy"]

    yavg = hst["Snet"].to_series()
    yavg = yavg.rolling(window, center=True, min_periods=1).mean()
    plt.plot(tMyr, yavg, color=colors_dark["Snet"], label=r"$\mathcal{L}-\mathcal{G}$")

    yavg = (
        hst["net"] - hst["SSN"] + hst["Fout"] - hst["Gwork"] - hst["Sbox"]
    ).to_series()
    yavg = yavg.rolling(window, center=True, min_periods=1).mean()
    plt.plot(
        tMyr,
        yavg,
        color=colors_dark["Srad"],
        ls=":",
        label=r"$(\mathcal{L}+\mathcal{F}_{\rm E, out}) -(\mathcal{G}+G_{xy}+R_{xy}+M_{xy})$",
    )

    yavg = (hst["SSN"] - hst["Fout"]).to_series()
    yavg = yavg.rolling(window, center=True, min_periods=1).mean()
    plt.plot(
        tMyr,
        yavg,
        color=colors_dark["SSN"],
        label=r"$\dot{S}_{\rm SN} - \mathcal{F}_{\rm E, out}$",
    )

    yavg = (hst["Gwork"] + hst["Sbox"]).to_series()
    yavg = yavg.rolling(window, center=True, min_periods=1).mean()
    plt.plot(tMyr, yavg, color=colors_dark["Sbox"], label=r"$G_{xy}+R_{xy}+M_{xy}$")

    yavg = hst["SKE"].to_series().rolling(window, center=True, min_periods=1).mean()
    plt.plot(tMyr, yavg, color=colors_dark["SKE"], label=r"$\dot{S}_{\rm diss}$")
    plt.fill_between(
        tMyr, yavg / 2, yavg * 2, color=colors_dark["SKE"], alpha=0.5, lw=0
    )
    plt.yscale("log")
    plt.ylim(1.0e-4, 1.0e2)

    plt.ylabel(r"Energy Source/Sink" + r"$\,[{L_\odot\,{\rm pc^{-2}}}]$")
    plt.setp(axes, xlabel=r"$t\,[{\rm Myr}]$")
    plt.tight_layout()
    lines = axes.lines

    labelLines(
        align=False,
        zorder=2.5,
    )

In [ ]:
for m in hst:
    plot_energetics_residual(hst[m])
    plt.title(m, fontsize="xx-small")
    plt.savefig(os.path.join(outdir, f"{m}-erate_tevol2.png"), dpi=300)